## Function to calculate BMI  z-score's for children 

LMS calculation for the BMI z-score 

Link to formulas used in the function: https://cdn.who.int/media/docs/default-source/child-growth/growth-reference-5-19-years/computation.pdf?sfvrsn=c2ff6a95_4 

All calculations are with age(months), weight(kg), height(cm)

You must have downloaded the reference sheets and not changed name on them



For the function to work your dataframe must be called "df". 

You must have following columns named in the correct format "age", "sex", "weight", "height". 

You must NOT have a column named "BMI" in your intial dataframe



The function will create three new columns: BMI, reference sheet used, and z-score



Now you can load the function and go! 



In [1]:
import pandas as pd

In [12]:
## Create WHO z-scores for children 0-19 years old
def zscore(df): 
    
    # create BMI column 
    weight = df['weight']
    height = (df['height']/100)
    BMI = []
    BMI = weight/(height**2)
    df['BMI'] = round(BMI,2)
     
    # this is the criteria for above/below 5 years in months (60 months = 5 years )
    age_5years = 60             

    # this is a for-loop to iterate through every single line of the dataframe 
    for index, row in df.iterrows():  

    # create new columns to save in 
        df.at[index, 'reference_sheet'] = ""
        df.at[index, 'zscore'] = ""

    # save parameters for calculations    
        reference_sheet = []
        age = row['age'] #
        sex = row['sex']
        BMI = row['BMI'] # this one is for the z-score calculation 

    # decide what reference material and prepare to get the values you need from it (boy/girl, above/below 5years)
        if age > age_5years:
            if sex == 'boy':
                reference = pd.read_csv("data/WHO boy reference above 5 years.csv")
                reference_sheet = ['Boy above 5']                         # Tell us what reference sheet I got

            else: 
                reference = pd.read_csv("WHO girl reference above 5 years.csv")
                reference_sheet = ['Girl above 5']


        else:
            if sex == 'boy':
                reference = pd.read_csv("WHO boy reference below 5 years.csv")
                reference_sheet = ["Boy below 5"]


            else:
                reference = pd.read_csv("WHO girl reference below 5 years.csv")
                reference_sheet = ["Girl below 5"]
                

    # Extract LMS values 
        individual_reference = reference[reference["Months"] == age]
        L = float(individual_reference["L"].iloc[0])              
        M = float(individual_reference["M"].iloc[0])
        S = float(individual_reference["S"].iloc[0])        
        SD3pos = round((M*((1+L*S*3)** (1 / L))),3)
        SD23pos= (M * ((1 + L * S * 3) ** (1 / L))) - (M * ((1 + L * S * 2) ** (1 / L)))
        SD3neg = round((M*((1+L*S*(-3))** (1 / L))),3)
        SD23neg= (M * ((1 + L * S * (-2)) ** (1 / L))) - (M * ((1 + L * S * (-3)) ** (1 / L)))

    # Calculate z-score 
        zscore = (((BMI/M)**L) -1)/(S*L)
        if zscore > 3:
            zscore == 3+((BMI-SD3pos)/SD23pos)
        if zscore <-3:
            zscore == -3+((BMI-SD3neg)/SD23neg)

            
    # save the reference sheet you used and the z-score you calculated 
        df.at[index, 'reference_sheet'] = reference_sheet
        df.at[index, 'zscore'] = zscore
    return df 

In [13]:
# run function 
zscore(df)

NameError: name 'df' is not defined